In [1]:
import pandas as pd
import glob

In [2]:
# function to process all individual csv annotations into one dataframe
def return_df_from_files(casualty_directory):
    df = pd.concat([pd.read_csv(f) for f in glob.glob(casualty_directory+'*.csv')])
    
    # make sure we do not have any duplicate sentences for same articles
    df.drop_duplicates(['article_id', 'sentence'], keep = 'first', inplace=True)
    
    return df
    
file_annotations = return_df_from_files('./fatality_counts/')

In [4]:
# otherwise we can load from saved annotations

annotations_articles = pd.read_csv('./fatality_counts/summary/annotations_articles_20231007-20231202.csv')
annotations_livefeeds = pd.read_csv('./fatality_counts/summary/annotations_livefeed_20231007-20231202.csv')

combined = pd.concat([annotations_articles, annotations_livefeeds, file_annotations])

#drop any duplicate annotations from files
combined.drop_duplicates(['article_title', 'sentence'], keep = 'first', inplace=True)

# set to datetime and filter for period of interest
combined['article_date'] = pd.to_datetime(combined['article_date'])
combined = combined[(combined['article_date'] > '2023-10-06')]

print(combined.shape)

(8068, 7)


In [5]:
summary = combined.groupby(['article_date', 'category']).agg({'article_id':'count'}).reset_index().pivot(index='article_date', columns='category', values='article_id').reset_index()

for column in ['both', 'israel', 'palestine']:
    if column not in summary:
        summary[column] = 0
    summary[column].fillna(0, inplace=True)

summary['both'].fillna(0, inplace=True)
summary['israel'] = summary['israel'] + summary['both']
summary['palestine'] = summary['palestine'] + summary['both']

summary.head(5)

category,article_date,both,israel,palestine
0,2023-10-07,2.0,37.0,23.0
1,2023-10-08,4.0,85.0,56.0
2,2023-10-09,10.0,104.0,49.0
3,2023-10-10,1.0,65.0,41.0
4,2023-10-11,11.0,113.0,60.0


In [6]:
summary_grouped = summary.groupby(pd.Grouper(key='article_date', freq='7D')).agg({'israel':'sum', 'palestine':'sum'}).reset_index()
summary_grouped = summary_grouped[summary_grouped['article_date'] < '2024-07-01']

summary_grouped.head(5)

category,article_date,israel,palestine
0,2023-10-07,509.0,306.0
1,2023-10-14,282.0,276.0
2,2023-10-21,205.0,320.0
3,2023-10-28,189.0,281.0
4,2023-11-04,164.0,322.0


In [183]:
import plotly.express as px

fig = px.line(summary_grouped, x="article_date", y=summary_grouped.columns[1:3])
fig.update_layout(
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    xaxis_title="",
    yaxis_title="",
    legend_title="",
    title="Mentions of Palestinian and Israeli Deaths by the BBC",
    template="presentation",
    plot_bgcolor="wheat",
    paper_bgcolor="wheat"
)
fig.show()